In [1]:
import os, sys, time
import tifffile as tif
sys.path.append("/home/emilyjanedennis/Desktop/GitHub/rat_BrainPipe")
from tools.utils.io import makedir
from tools.registration.register import change_interpolation_order, transformix_command_line_call
from tools.registration.transform_list_of_points import modify_transform_files
from scipy.ndimage.interpolation import zoom
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv

In [2]:
src = "/home/emilyjanedennis/Desktop/for_registration_to_lightsheet"
mouse_anns_in_rat = os.path.join(src, "output_dirs/PMA_ann_in_PRA_25/PMA_ann_in_PRA.tif")
mouse_anns_jacobian_filename = os.path.join(src, "output_dirs/PMA_ann_in_PRA_25/PMA_ann_in_PRA_spatialJacobian.tif")
mouse_anns_net_jacobian_filename = os.path.join(src, "tiffs/PMA_in_PRA_adj_jacobian_16.tif")

rat_anns_filename = os.path.join(src,"labels/SIGMA_labels.csv") 
mouse_anns_filename = os.path.join(src,"labels/PMA_ann.csv")
savefile= "/home/emilyjanedennis/Desktop/mouserat.tif"
savecsv="/home/emilyjanedennis/Desktop/mouserat_percent.csv"

### to do
want to compare (1) mouse and rat annotations when in the same space (for now, mouse in rat space, want to do reverse too, and individual brains)

segregate mouse jacobian data into rat and mouse regions and see which are biggest contributors 

In [30]:
mouse_ann_tiff = tif.imread(mouse_anns_in_rat)
print('ann import')
mouse_jacobian = tif.imread(mouse_anns_jacobian_filename)
print('jac import')
mouse_net_jacobian = tif.imread(mouse_anns_net_jacobian_filename)
print('net jac import')
mouse_ann_original = tif.imread('/home/emilyjanedennis/Desktop/for_registration_to_lightsheet/tiffs/PMA_annotation.tif')
mouse_ann_affine = tif.imread('/home/emilyjanedennis/Desktop/for_registration_to_lightsheet/output_dirs/PMA_ann_in_PRA_25_affine/result.tif')
rat_SIGMA = tif.imread('/home/emilyjanedennis/Desktop/for_registration_to_lightsheet/output_dirs/SIGMA_ann_in_PRA_25/result.tif')


ann import


TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'


jac import
net jac import


In [10]:
rat_SIGMA_anns = ('/home/emilyjanedennis/Desktop/for_registration_to_lightsheet/labels/SIGMA_labels.csv')
np.shape(rat_SIGMA)

(618, 1150, 385)

In [11]:
print(np.shape(mouse_ann_tiff))
print(np.shape(mouse_jacobian))
print(np.shape(mouse_net_jacobian))


(618, 1150, 385)
(618, 1150, 385)
(618, 1150, 385)


In [25]:
# make sure your annotations have the first column as id number, 2nd as abbreviation, or else change the input below
mouseanns=pd.read_csv(mouse_anns_filename,usecols=[0,1,2],header=0)
ratanns = pd.read_csv(rat_SIGMA_anns)
print("read annotations")

read annotations


In [26]:
dfm=mouseanns.dropna()
dfr = ratanns.dropna()

In [29]:
dfm.columns = ['ID','abbr','fullname']

In [31]:
dfr.insert(3,'PMA',0)
dfr.insert(4,'PMA_affine',0)
dfr.insert(5,'PMA_in_PRA',0)
dfr.insert(6,'PRA',0)

In [32]:
dfm.insert(3,'PMA',0)
dfm.insert(4,'PMA_affine',0)
dfm.insert(5,'PMA_in_PRA',0)
dfm.insert(6,'PRA',0)

In [35]:
for i in dfm['ID']:
    dfm.PMA[dfm.index[dfm.ID==i]] = np.size(mouse_ann_original[mouse_ann_original==i])
    dfm.PMA_affine[dfm.index[dfm.ID==i]] = np.size(mouse_ann_affine[mouse_ann_affine==i])
    dfm.PMA_in_PRA[dfm.index[dfm.ID==i]] = np.size(mouse_ann_tiff[mouse_ann_tiff==i])

dfm.to_csv('/home/emilyjanedennis/Desktop/mouse_anns_in_PRA.csv')

<ipython-input-35-be9e2f7c077a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfm.PMA[dfm.index[dfm.ID==i]] = np.size(mouse_ann_original[mouse_ann_original==i])
/home/emilyjanedennis/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:1295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_labels(key, value)
/home/emilyjanedennis/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

In [ ]:
dfm.to_csv('/home/emilyjanedennis/Desktop/mouse_anns_in_PRA.csv')

In [36]:
mouse_ann_tiff = tif.imread(mouse_anns_in_rat)
print('ann import')
mouse_jacobian = tif.imread(mouse_anns_jacobian_filename)
print('jac import')
mouse_net_jacobian = tif.imread(mouse_anns_net_jacobian_filename)
print('net jac import')
mouse_ann_original = tif.imread('/home/emilyjanedennis/Desktop/for_registration_to_lightsheet/tiffs/PMA_annotation.tif')
mouse_ann_affine = tif.imread('/home/emilyjanedennis/Desktop/for_registration_to_lightsheet/output_dirs/PMA_ann_in_PRA_25_affine/result.tif')
rat_SIGMA = tif.imread('/home/emilyjanedennis/Desktop/for_registration_to_lightsheet/output_dirs/SIGMA_ann_in_PRA_25/result.tif')


ann import


TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'


jac import
net jac import


In [37]:
# segment by SIGMA or PMA region
# compare jacobian results 
sigma = rat_SIGMA
mouse = mouse_ann_tiff 

for i in np.unique(sigma):
    sigma[rat_SIGMA==i] = np.mean(mouse_net_jacobian[rat_SIGMA==i])

tif.imsave('/home/emilyjanedennis/Desktop/sigma_mouse_jacobian.tif',sigma)

for i in np.unique(mouse):
    mouse[mouse_ann_tiff == i] = np.mean(mouse_net_jacobian[mouse_ann_tiff==i])

tif.imsave('/home/emilyjanedennis/Desktop/PMA_mouse_jacobian.tif',mouse)